<a href="https://colab.research.google.com/github/MoonJaeHoon/dacon_parking_demand/blob/master/LBscore_110(Lasso%26Add2Feature_from_external).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 주제: 유형별 임대주택 설계 시 단지 내 적정 주차 수요를 예측

+ 아파트 단지 내 필요한 주차대수는 ①법정주차대수 ②장래주차수요 중 큰 값에 따라 결정하게되어 있어, 정확한 ②장래주차수요의 산정을 필요로 함
+ 현재 ②장래주차수요는 ‘주차원단위’와 ‘건축연면적’을 기초로하여 산출되고 있으며, ‘주차원단위’는 신규 건축예정 부지 인근의 유사 단지를 피크 시간대 방문하여 주차된 차량대수를 세는 방법으로 조사하고 있음
+ 이 경우 인력조사로 인한 오차발생, 현장조사 시점과 실제 건축시점과의 시간차 등의 문제로 과대 또는 과소 산정의 가능성을 배제할 수 없음

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import os
import random
import time
from datetime import timedelta, timezone, datetime
from copy import deepcopy
from collections import defaultdict
# 한글폰트 설정, 그래프 마이너스 표시 설정
import matplotlib
from matplotlib import font_manager, rc
from matplotlib import pyplot as plt
plt.rcParams['font.family']=['NanumGothic', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
import platform
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

seed=47
def set_seed(seed: int = 42, contain_cuda: bool = False):
    random.seed(seed)
    np.random.seed(seed)

    # if contain_cuda:
    #     torch.backends.cudnn.deterministic = True
    #     torch.backends.cudnn.benchmark = False

    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)
    print(f"Seed set as {seed}")
set_seed(seed)

Seed set as 47


## 1. 데이터 탐색

### 1.1. 데이터 불러오기 & 간단한 탐색

In [ ]:
cd /content/drive/MyDrive/dacon_parking_data

/content/drive/MyDrive/dacon_parking_data


In [ ]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
age_gender_info = pd.read_csv('./data/age_gender_info.csv')

In [ ]:
# 결측치가 1개라도 존재하는 row들 & 결측치가 전혀 없었던 단지코드들
print(f"train.shape : {train.shape}")
exist_nan_row_idx = train.isnull()[train.isnull().any(axis=1)].index
print(f"len of exist_nan_row_idx : {len(exist_nan_row_idx)}")
no_nan_danjicode = list(set(train['단지코드']) - set(train.loc[exist_nan_row_idx,:]['단지코드']))
print(f"len of exist_nan_danjicode : {len(set(train['단지코드']))-len(no_nan_danjicode)}")
print('='*30)
print(f"len of no_nan_danjicode : {len(no_nan_danjicode)}")
print(no_nan_danjicode)

train.shape : (2952, 15)
len of exist_nan_row_idx : 646
len of exist_nan_danjicode : 45
len of no_nan_danjicode : 378
['C2316', 'C2001', 'C1802', 'C2536', 'C1850', 'C1968', 'C1970', 'C2560', 'C1329', 'C2274', 'C1592', 'C2351', 'C2479', 'C1171', 'C1103', 'C2663', 'C2032', 'C2490', 'C1527', 'C1880', 'C1936', 'C1032', 'C1129', 'C2225', 'C1569', 'C1244', 'C1519', 'C2596', 'C2358', 'C1343', 'C1889', 'C2329', 'C1155', 'C1077', 'C1022', 'C1213', 'C1673', 'C2008', 'C2368', 'C1666', 'C1819', 'C1879', 'C2437', 'C2086', 'C1227', 'C2340', 'C2412', 'C2522', 'C2361', 'C1656', 'C2035', 'C2000', 'C2040', 'C2068', 'C1428', 'C1263', 'C1368', 'C1281', 'C2661', 'C1470', 'C2046', 'C2579', 'C2154', 'C2635', 'C1973', 'C1638', 'C2563', 'C1487', 'C1286', 'C1049', 'C1969', 'C1122', 'C2049', 'C2188', 'C2141', 'C1985', 'C2262', 'C2010', 'C1945', 'C1218', 'C1065', 'C1057', 'C1310', 'C2514', 'C2153', 'C1668', 'C1786', 'C2489', 'C1699', 'C1740', 'C1907', 'C2245', 'C2659', 'C2680', 'C1000', 'C2325', 'C2513', 'C1677',

In [ ]:
test.isnull().sum().to_dict()

{'공가수': 0,
 '공급유형': 0,
 '단지내주차면수': 0,
 '단지코드': 0,
 '도보 10분거리 내 버스정류장 수': 0,
 '도보 10분거리 내 지하철역 수(환승노선 수 반영)': 42,
 '임대건물구분': 0,
 '임대료': 180,
 '임대보증금': 180,
 '자격유형': 2,
 '전용면적': 0,
 '전용면적별세대수': 0,
 '지역': 0,
 '총세대수': 0}

+ train 데이터에서는 임대보증금, 임대료, 도보 10분거리 내 지하철역 수(환승노선 수 반영), 도보 10분거리 내 버스정류장 수에 NULL값 존재

+ test 데이터에서는 자격유형, 임대보증금, 임대료, 도보 10분거리 내 지하철역 수(환승노선 수 반영)에 NULL값 존재

In [ ]:
# 임대보증금과 임대료는 '-'을 null로 바꾼 후 float로 타입 변경
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
test['임대보증금'] = test['임대보증금'].astype(float)

train.loc[train.임대료=='-', '임대료'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
train['임대료'] = train['임대료'].astype(float)
test['임대료'] = test['임대료'].astype(float)

### 1.2. NULL 값이 있는 변수 탐색

#### (1) 임대보증금, 임대료

In [ ]:
train[train.임대보증금.isnull()].공급유형.value_counts()

임대상가    562
국민임대      8
공공분양      7
행복주택      4
Name: 공급유형, dtype: int64

In [ ]:
test[test.임대보증금.isnull()].공급유형.value_counts()

임대상가    177
영구임대      5
행복주택      4
Name: 공급유형, dtype: int64

In [ ]:
train[train.임대보증금.isnull()].자격유형.value_counts()

D    569
H      8
K      4
Name: 자격유형, dtype: int64

In [ ]:
test[test.임대보증금.isnull()].자격유형.value_counts()

D    180
L      4
C      2
Name: 자격유형, dtype: int64

In [ ]:
train[train.자격유형.isin(['D', 'H', 'K'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 757 entries, 88 to 2919
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          757 non-null    object 
 1   총세대수                          757 non-null    int64  
 2   임대건물구분                        757 non-null    object 
 3   지역                            757 non-null    object 
 4   공급유형                          757 non-null    object 
 5   전용면적                          757 non-null    float64
 6   전용면적별세대수                      757 non-null    int64  
 7   공가수                           757 non-null    float64
 8   자격유형                          757 non-null    object 
 9   임대보증금                         176 non-null    float64
 10  임대료                           176 non-null    float64
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  623 non-null    float64
 12  도보 10분거리 내 버스정류장 수            757 non-null    float64
 13  단지내

In [ ]:
test[test.자격유형.isin(['D', 'L', 'C'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226 entries, 73 to 1017
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          226 non-null    object 
 1   총세대수                          226 non-null    int64  
 2   임대건물구분                        226 non-null    object 
 3   지역                            226 non-null    object 
 4   공급유형                          226 non-null    object 
 5   전용면적                          226 non-null    float64
 6   전용면적별세대수                      226 non-null    int64  
 7   공가수                           226 non-null    float64
 8   자격유형                          226 non-null    object 
 9   임대보증금                         40 non-null     float64
 10  임대료                           40 non-null     float64
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  201 non-null    float64
 12  도보 10분거리 내 버스정류장 수            226 non-null    float64
 13  단지내

In [ ]:
# 임대보증금과 임대료가 NULL인 경우는 0으로 대체하는 것이 좋아보임
train[['임대보증금', '임대료']] = train[['임대보증금', '임대료']].fillna(0)
test[['임대보증금', '임대료']] = test[['임대보증금', '임대료']].fillna(0)

#### (2) 도보 10분거리 내 지하철역 수(환승노선 수 반영), 도보 10분거리 내 버스정류장 수

+ NULL 값을 0으로 대체

In [ ]:
subway_null_codes = train[train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()].단지코드.unique()
train.loc[train.단지코드.isin(subway_null_codes), '도보 10분거리 내 지하철역 수(환승노선 수 반영)'].value_counts(dropna=False)

NaN    211
Name: 도보 10분거리 내 지하철역 수(환승노선 수 반영), dtype: int64

In [ ]:
subway_null_codes = test[test['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()].단지코드.unique()
test.loc[test.단지코드.isin(subway_null_codes), '도보 10분거리 내 지하철역 수(환승노선 수 반영)'].value_counts(dropna=False)

NaN    42
Name: 도보 10분거리 내 지하철역 수(환승노선 수 반영), dtype: int64

In [ ]:
bus_null_codes = train[train['도보 10분거리 내 버스정류장 수'].isnull()].단지코드.unique()
train.loc[train.단지코드.isin(bus_null_codes), '도보 10분거리 내 버스정류장 수'].value_counts(dropna=False)

NaN    4
Name: 도보 10분거리 내 버스정류장 수, dtype: int64

In [ ]:
cols = ['도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수']
train[cols] = train[cols].fillna(0)
test[cols] = test[cols].fillna(0)

#### (3) 자격유형

In [ ]:
test[test.자격유형.isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000.0,37470.0,0.0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000.0,44770.0,0.0,2.0,173.0


In [ ]:
test[test.단지코드=='C2411'] # A로 채우면 될듯

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
193,C2411,962,아파트,경상남도,국민임대,39.43,56,25.0,A,11992000.0,100720.0,0.0,2.0,840.0
194,C2411,962,아파트,경상남도,국민임대,39.72,336,25.0,A,11992000.0,100720.0,0.0,2.0,840.0
195,C2411,962,아파트,경상남도,국민임대,39.82,179,25.0,A,11992000.0,100720.0,0.0,2.0,840.0
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000.0,37470.0,0.0,2.0,840.0
197,C2411,962,아파트,경상남도,국민임대,51.93,150,25.0,A,21586000.0,171480.0,0.0,2.0,840.0


In [ ]:
test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'

In [ ]:
test[test.단지코드=='C2253'].head() # 임대보증금과 임대료가 존재하는 경우 자격유형이 C => C로 채우면 될듯

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000.0,44770.0,0.0,2.0,173.0
259,C2253,1161,아파트,강원도,영구임대,31.32,239,0.0,C,3731000.0,83020.0,0.0,2.0,173.0
260,C2253,1161,아파트,강원도,영구임대,31.32,149,0.0,C,3731000.0,83020.0,0.0,2.0,173.0
261,C2253,1161,상가,강원도,임대상가,13.77,1,0.0,D,0.0,0.0,0.0,2.0,173.0
262,C2253,1161,상가,강원도,임대상가,22.89,1,0.0,D,0.0,0.0,0.0,2.0,173.0


In [ ]:
test.loc[test.단지코드.isin(['C2253']) & test.자격유형.isnull(), '자격유형'] = 'C'

### 1.3. 중복확인

In [ ]:
train.shape, train.drop_duplicates().shape # 중복 존재

((2952, 15), (2632, 15))

In [ ]:
test.shape, test.drop_duplicates().shape # 중복 존재

((1022, 14), (949, 14))

+ 중복되는 데이터는 제거하고 분석을 진행하는 것이 좋아보임

In [ ]:
train = train.drop_duplicates()
test = test.drop_duplicates()

In [ ]:
train[train.단지코드=='C2483']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000.0,103680.0,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000.0,103680.0,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
5,C2483,900,아파트,경상북도,국민임대,51.95,89,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
6,C2483,900,아파트,경상북도,국민임대,51.95,135,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
7,C2483,900,아파트,경상북도,국민임대,59.88,86,38.0,A,30357000.0,214270.0,0.0,3.0,1425.0,1015.0


In [ ]:
print(f"단지코드 C2483에서 유일한 값을 가지는 변수들:\n{list(train.columns[train[train.단지코드=='C2483'].nunique()==1])}")

단지코드 C2483에서 유일한 값을 가지는 변수들:
['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '공가수', '자격유형', '도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수']


In [ ]:
train.groupby(['단지코드']).nunique(dropna=False)

,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
단지코드,,,,,,,,,,,,,,
C1000,1,1,1,1,5,6,1,1,3,3,1,1,1,1
C1004,1,2,1,2,15,3,1,2,3,3,1,1,1,1
C1005,1,1,1,1,3,3,1,1,3,3,1,1,1,1
C1013,1,1,1,1,4,5,1,1,3,3,1,1,1,1
C1014,1,1,1,1,6,7,1,1,4,4,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2663,1,1,1,1,5,6,1,1,4,4,1,1,1,1
C2666,1,1,1,1,4,6,1,1,2,2,1,1,1,1
C2670,1,1,1,1,4,3,1,1,3,3,1,1,1,1


In [ ]:
train.groupby(['단지코드']).nunique(dropna=False).sum(axis=0)

총세대수                             423
임대건물구분                           456
지역                               423
공급유형                             488
전용면적                            1898
전용면적별세대수                        2230
공가수                              423
자격유형                             510
임대보증금                           1277
임대료                             1289
도보 10분거리 내 지하철역 수(환승노선 수 반영)     423
도보 10분거리 내 버스정류장 수               423
단지내주차면수                          423
등록차량수                            423
dtype: int64

+ 값이 423보다 크면 하나의 단지코드에 대해 둘 이상의 항목이 존재하는 것
+ 임대건물구분, 공급유형, 전용면적, 전용면적별세대수, 자격유형, 임대보증금, 임대료는 하나의 단지코드에 대해 둘 이상의 항목 존재
+ 단지코드별 집계시 총세대수, 지역, 공가수, 도보 10분거리 내 지하철역 수(환승노선 수 반영), 도보 10분거리 내 버스정류장 수, 단지내주차면수, 등록차량수는 그대로 사용하고, 나머지 변수들은 각 항목들을 변수로 만들어 사용하는 것이 좋아보임

## 1.4 단지코드 별 집계

#### (1) 하나의 단지코드에 하나의 값만 존재하는 변수들

+ 총세대수, 지역, 공가수, 도보 10분거리 내 지하철역 수(환승노선 수 반영), 도보 10분거리 내 버스정류장 수, 단지내주차면수, 등록차량수

In [ ]:
unique_cols = ['총세대수', '지역', '공가수', 
               '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
               '도보 10분거리 내 버스정류장 수',
               '단지내주차면수', '등록차량수']
train_agg = train.set_index('단지코드')[unique_cols].drop_duplicates()
test_agg = test.set_index('단지코드')[[col for col in unique_cols if col!='등록차량수']].drop_duplicates()

In [ ]:
train_agg

,총세대수,지역,공가수,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
단지코드,,,,,,,
C2483,900,경상북도,38.0,0.0,3.0,1425.0,1015.0
C2515,545,경상남도,17.0,0.0,3.0,624.0,205.0
C1407,1216,대전광역시,13.0,1.0,1.0,1285.0,1064.0
C1945,755,경기도,6.0,1.0,3.0,734.0,730.0
C1470,696,전라북도,14.0,0.0,2.0,645.0,553.0
...,...,...,...,...,...,...,...
C2586,90,제주특별자치도,7.0,0.0,3.0,66.0,57.0
C2035,492,강원도,24.0,0.0,1.0,521.0,246.0
C2020,40,부산광역시,7.0,1.0,2.0,25.0,19.0


In [ ]:
test_agg

,총세대수,지역,공가수,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
단지코드,,,,,,
C1072,754,경기도,14.0,0.0,2.0,683.0
C1128,1354,경기도,9.0,0.0,3.0,1216.0
C1456,619,부산광역시,18.0,0.0,16.0,547.0
C1840,593,전라북도,7.0,0.0,3.0,543.0
C1332,1297,경기도,11.0,0.0,2.0,1112.0
...,...,...,...,...,...,...
C2456,349,제주특별자치도,17.0,0.0,4.0,270.0
C1266,596,충청북도,35.0,0.0,1.0,593.0
C2152,120,강원도,9.0,0.0,1.0,40.0


#### (2) 하나의 단지코드에 둘 이상의 값이 존재하는 변수들

+ 임대건물구분, 공급유형, 전용면적, 전용면적별세대수, 자격유형, 임대보증금, 임대료

In [ ]:
def reshape_cat_features(data, cast_col, value_col):
    res = data.drop_duplicates(['단지코드', cast_col]).assign(counter=1).pivot(index='단지코드', columns=cast_col, values=value_col).fillna(0)
    res.columns.name = None
    res = res.rename(columns={col:cast_col+'_'+col for col in res.columns})
    return res

#### 1) 임대건물구분: 아파트/상가 

+ 주상복합인 경우 주차수요가 주거only인 경우보다 많지 않을까? => 상가 직원들도 차량 등록이 가능하면 상가 당 주차수요는 일반 가구보다 높을  수 있음
+ 상가의 전용면적별세대수는 모두 1
+ 우선 임대건물구분을 열로 올려서 단지가 주상복합인지 아닌지 구분할 수 있게 해야함

In [ ]:
reshape_cat_features(data=train, cast_col='임대건물구분', value_col='counter')

,임대건물구분_상가,임대건물구분_아파트
단지코드,,
C1000,0.0,1.0
C1004,1.0,1.0
C1005,0.0,1.0
C1013,0.0,1.0
C1014,0.0,1.0
...,...,...
C2663,0.0,1.0
C2666,0.0,1.0
C2670,0.0,1.0


In [ ]:
reshape_cat_features(data=test, cast_col='임대건물구분', value_col='counter')

,임대건물구분_상가,임대건물구분_아파트
단지코드,,
C1003,0.0,1.0
C1006,1.0,1.0
C1016,0.0,1.0
C1019,0.0,1.0
C1030,0.0,1.0
...,...,...
C2653,0.0,1.0
C2675,0.0,1.0
C2676,1.0,1.0


#### 공급유형

+ [국민임대와 공공임대의 차이](https://brunch.co.kr/@leeeeesh/91)
+ LH공사와 SH공사에서 제공하는 국민임대는 국가 재정과 국민주택기금을 지원받아 국가, 지방자치단체, 한국토지주택공사 또는 지방공사가 건설, 공급하는 주택을 의미한다. 저렴한 임대료를 지불하고 장기(30년) 임대가 가능하다. 장기 거주는 가능하지만 해당 주거지를 매입할 수 없다는 단점이 있다.
+ 반면 공공임대는 5년 또는 10년 후 임대기간이 종료되었을 때 입주자에게 우선 분양전환혜택을 준다.

In [ ]:
pd.concat([train.공급유형.value_counts(), test.공급유형.value_counts()], axis=1)

,공급유형,공급유형
국민임대,1730,619.0
임대상가,285,112.0
공공임대(10년),203,34.0
행복주택,203,121.0
영구임대,149,44.0
공공임대(50년),31,13.0
공공임대(분납),12,6.0
장기전세,9,NaN
공공분양,7,NaN
공공임대(5년),3,NaN


In [ ]:
train.loc[train.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
test.loc[test.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
train.loc[train.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'
test.loc[test.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'

In [ ]:
set(train.공급유형)

{'공공임대(50년)', '공공임대(5년/10년/분납/분양)', '국민임대/장기전세', '영구임대', '임대상가', '행복주택'}

In [ ]:
set(test.공급유형)

{'공공임대(50년)', '공공임대(5년/10년/분납/분양)', '국민임대/장기전세', '영구임대', '임대상가', '행복주택'}

In [ ]:
reshape_cat_features(data=train, cast_col='공급유형', value_col='counter')

,공급유형_공공임대(50년),공급유형_공공임대(5년/10년/분납/분양),공급유형_국민임대/장기전세,공급유형_영구임대,공급유형_임대상가,공급유형_행복주택
단지코드,,,,,,
C1000,0.0,0.0,1.0,0.0,0.0,0.0
C1004,0.0,0.0,0.0,1.0,1.0,0.0
C1005,0.0,0.0,1.0,0.0,0.0,0.0
C1013,0.0,0.0,1.0,0.0,0.0,0.0
C1014,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
C2663,0.0,0.0,1.0,0.0,0.0,0.0
C2666,0.0,0.0,1.0,0.0,0.0,0.0
C2670,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
reshape_cat_features(data=test, cast_col='공급유형', value_col='counter')

,공급유형_공공임대(50년),공급유형_공공임대(5년/10년/분납/분양),공급유형_국민임대/장기전세,공급유형_영구임대,공급유형_임대상가,공급유형_행복주택
단지코드,,,,,,
C1003,0.0,0.0,0.0,0.0,0.0,1.0
C1006,0.0,0.0,0.0,1.0,1.0,0.0
C1016,0.0,0.0,1.0,0.0,0.0,0.0
C1019,0.0,0.0,1.0,0.0,0.0,0.0
C1030,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
C2653,0.0,0.0,1.0,0.0,0.0,0.0
C2675,0.0,0.0,1.0,0.0,0.0,0.0
C2676,0.0,0.0,0.0,1.0,1.0,0.0


#### 자격유형

+ 변수생성 컨셉: 특정자격 유형이 많으면 그 단지는 등록차량수가 적을 것이다.
    + 예를들어, 소득수준이 낮은 자격유형의 세대가 많으면 주차수요가 적을 가능성이 높을 것으로 예상해볼 수 있다.
+ 문제는 특성이 비슷한 항목끼리 묶어야 의미가 있을 것 같은데, 비식별화 되어 있어서 각 코드값이 무엇인지 정확히 알 수 없다는 것임
+ 단지코드별로 자격유형별 세대수를 알 수 있으면 좋을 것 같지만 이 데이터에서는 파악 불가능
+ 자격유형별 소득수준을 간접적으로 파악하기 위해 자격유형별 임대보증금의 평균, 임대료의 평균을 변수로 사용해 볼 수 있을 것 같음

In [ ]:
pd.concat([train.자격유형.value_counts(), 
           test.자격유형.value_counts()], axis=1)

,자격유형,자격유형
A,1775,569.0
D,292,114.0
H,154,92.0
J,105,81.0
C,92,35.0
I,49,7.0
E,37,10.0
L,33,12.0
K,33,16.0
N,29,10.0


+ B, F, O가 train에만 있고 test에는 없음

+ [공공임대주택 공급대상](https://www.myhome.go.kr/hws/portal/cont/selectContRentalView.do#guide=RH101)
    + 영구임대: 생계급여 또는 의료급여 수급자 등[소득 1분위] - 1
    + 국민임대: 무주택세대구성원[소득 2~4분위] - 3
    + 장기전세: 무주택세대구성원[소득 3~4분위] - 2
    + 공공임대(5년/10년/분납): 무주택세대구성원[소득 3~5분위] - 3 
    + 행복주택: 무주택세대구성원/무주택자[소득 2~5분위] - 4
    + 이렇게 나누면 13개 항목인데 자격유형의 항목수는 15개

In [ ]:
train.loc[train.자격유형=='B', ['임대건물구분', '공급유형']].drop_duplicates()
# 자격유형 B는 공급유형이 모두 국민임대/장기전세, 공급유형이 국민임대/장기전세 일때 자격유형이 어떤게 있는지 보는게 좋을 듯

,임대건물구분,공급유형
26,아파트,국민임대/장기전세


In [ ]:
train.loc[train.공급유형.isin(['국민임대/장기전세']), '자격유형'].value_counts()

A    1524
H     154
E      34
B      18
G       9
Name: 자격유형, dtype: int64

국민임대/장기전세

+ (일반)해당지역 거주 무주택세대구성원
+ (특별/우선)3자녀 이상 가구
+ (특별/우선)국가유공자
+ (특별/우선)영구임대 입주자
+ (특별/우선)비닐간이공작물 거주자
+ (특별/우선)신혼부부(혼인기간 5년이내)
+ (특별/우선)사업지구 철거민
+ (특별/우선)기타 공급대상(고령자, 노부모부양자, 장애인, 파독근로자 등)
+ 건수가 가장 많은 A는 해당지역 거주 무주택세대구성원으로 예상됨

In [ ]:
# 다른 공급유형별 자격유형도 살펴보자
train.loc[train.공급유형.isin(['영구임대']), '자격유형'].value_counts()

C    92
I    49
F     3
E     3
A     2
Name: 자격유형, dtype: int64

영구임대

+ (일반)생계급여 또는 의료급여 수급자
+ (일반)국가유공자
+ (특별/우선)수급자 선정기준의 소득인정액 이하인 국가유공자
+ (특별/우선)귀환국군포로
+ (특별/우선)수급자 신혼부부

In [ ]:
train.loc[train.공급유형.isin(['공공임대(5년/10년/분납/분양)']), '자격유형'].value_counts()

A    218
D      7
Name: 자격유형, dtype: int64

공공임대(5년/10년/분납/분양)

+ (일반)해당지역 거주무주택세대구성원
+ (특별/우선)다자녀 특별
+ (특별/우선)신혼부부 특별
+ (특별/우선)생애최초 특별
+ (특별/우선)노부모부양 특별
+ (특별/우선)국가유공자 특별
+ (특별/우선)기타 특별(장애인, 철거민 등)
+ A는 '(일반)해당지역 거주무주택세대구성원'으로 추정됨

In [ ]:
train.loc[train.공급유형.isin(['행복주택']), '자격유형'].value_counts()

J    105
K     33
L     33
N     29
M      2
O      1
Name: 자격유형, dtype: int64

행복주택

+ 대학생(취준생 포함), 사회초년생(제취준생 포함), 신혼부부(예비신혼부부, 대학생, 취준생 신혼부부 포함), 고령자, 주거급여수급자, 산업단지 근로자
+ 이렇게 6개 항목이랑 매핑 될듯

In [ ]:
train.loc[train.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '공급유형'].value_counts()
# 'J', 'L', 'K', 'N', 'M', 'O' 는 공급유형이 행복주택인 경우에서만 나타남 이것만 따로 묶는게 좋을듯

행복주택    203
Name: 공급유형, dtype: int64

In [ ]:
train.loc[train.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'
test.loc[test.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'

In [ ]:
train.loc[train.자격유형.isin(['A']), '공급유형'].value_counts()

국민임대/장기전세             1524
공공임대(5년/10년/분납/분양)     218
공공임대(50년)               31
영구임대                     2
Name: 공급유형, dtype: int64

In [ ]:
train.loc[train.자격유형.isin(['D']), '공급유형'].value_counts()

임대상가                  285
공공임대(5년/10년/분납/분양)      7
Name: 공급유형, dtype: int64

In [ ]:
train.loc[train.자격유형.isin(['H']), '공급유형'].value_counts()

국민임대/장기전세    154
Name: 공급유형, dtype: int64

In [ ]:
train.loc[train.자격유형.isin(['C']), '공급유형'].value_counts()

영구임대    92
Name: 공급유형, dtype: int64

In [ ]:
train.loc[train.자격유형.isin(['I']), '공급유형'].value_counts()

영구임대    49
Name: 공급유형, dtype: int64

In [ ]:
train.loc[train.자격유형.isin(['E']), '공급유형'].value_counts()

국민임대/장기전세    34
영구임대          3
Name: 공급유형, dtype: int64

In [ ]:
train.loc[train.자격유형.isin(['B']), '공급유형'].value_counts()

국민임대/장기전세    18
Name: 공급유형, dtype: int64

In [ ]:
train.loc[train.자격유형.isin(['G']), '공급유형'].value_counts()

국민임대/장기전세    9
Name: 공급유형, dtype: int64

In [ ]:
train.loc[train.자격유형.isin(['F']), '공급유형'].value_counts()

영구임대    3
Name: 공급유형, dtype: int64

+ 'H', 'B', 'E', 'G' 는 국민임대/장기전세 공급대상(E는 영구임대인 경우도 있긴 하지만 국민임대/장기전세인 경우가 더 많아서 여기에 포함시킴)
+ 'C', 'I', 'F' 는 영구임대 공급대상으로 묶어 보겠음

In [ ]:
train.loc[train.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'
test.loc[test.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'

train.loc[train.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'
test.loc[test.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'

In [ ]:
pd.concat([train.자격유형.value_counts(), 
           test.자격유형.value_counts()], axis=1)

,자격유형,자격유형
A,1775,569
D,292,114
국민임대/장기전세_공급대상,218,103
행복주택_공급대상,203,121
영구임대_공급대상,144,42


In [ ]:
reshape_cat_features(data=train, cast_col='자격유형', value_col='counter')

,자격유형_A,자격유형_D,자격유형_국민임대/장기전세_공급대상,자격유형_영구임대_공급대상,자격유형_행복주택_공급대상
단지코드,,,,,
C1000,1.0,0.0,0.0,0.0,0.0
C1004,0.0,1.0,0.0,1.0,0.0
C1005,1.0,0.0,0.0,0.0,0.0
C1013,1.0,0.0,0.0,0.0,0.0
C1014,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
C2663,0.0,0.0,1.0,0.0,0.0
C2666,1.0,0.0,0.0,0.0,0.0
C2670,1.0,0.0,0.0,0.0,0.0


In [ ]:
reshape_cat_features(data=train, cast_col='공급유형', value_col='counter')

,공급유형_공공임대(50년),공급유형_공공임대(5년/10년/분납/분양),공급유형_국민임대/장기전세,공급유형_영구임대,공급유형_임대상가,공급유형_행복주택
단지코드,,,,,,
C1000,0.0,0.0,1.0,0.0,0.0,0.0
C1004,0.0,0.0,0.0,1.0,1.0,0.0
C1005,0.0,0.0,1.0,0.0,0.0,0.0
C1013,0.0,0.0,1.0,0.0,0.0,0.0
C1014,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
C2663,0.0,0.0,1.0,0.0,0.0,0.0
C2666,0.0,0.0,1.0,0.0,0.0,0.0
C2670,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
print(train_agg.shape)
train_agg = pd.concat([train_agg,
                       reshape_cat_features(data=train, cast_col='임대건물구분', value_col='counter'),
                       reshape_cat_features(data=train, cast_col='공급유형', value_col='counter'),
                       reshape_cat_features(data=train, cast_col='자격유형', value_col='counter')], axis=1)

print(test_agg.shape)
test_agg = pd.concat([test_agg,
                       reshape_cat_features(data=test, cast_col='임대건물구분', value_col='counter'),
                       reshape_cat_features(data=test, cast_col='공급유형', value_col='counter'),
                       reshape_cat_features(data=test, cast_col='자격유형', value_col='counter')], axis=1)

(423, 7)
(150, 6)


In [ ]:
train_agg.shape, test_agg.shape

((423, 20), (150, 19))

In [ ]:
final_age_gender_info = pd.read_csv('./data/final_age_gender_info.csv',index_col=0)
final_age_gender_info_dict = final_age_gender_info['지역별차량등록대수(연령분포고려)'].to_dict()
train_agg['지역별차량등록대수(연령분포고려)'] = train_agg['지역'].map(final_age_gender_info_dict)
test_agg['지역별차량등록대수(연령분포고려)'] = test_agg['지역'].map(final_age_gender_info_dict)

In [ ]:
total_num_saedae_per_sido_dict = train_agg.groupby(['지역'])['총세대수'].sum().to_dict()
train_agg['단지코드내차량등록대수'] = train_agg['지역별차량등록대수(연령분포고려)'] * train_agg['총세대수'] / train_agg['지역'].map(total_num_saedae_per_sido_dict)
total_num_saedae_per_sido_dict = test_agg.groupby(['지역'])['총세대수'].sum().to_dict()
test_agg['단지코드내차량등록대수'] = test_agg['지역별차량등록대수(연령분포고려)'] * test_agg['총세대수'] / test_agg['지역'].map(total_num_saedae_per_sido_dict)
train_agg

In [ ]:
train_agg.shape, test_agg.shape

((423, 22), (150, 21))

# 2. Pycaret 모델링

In [ ]:
cat_features = ['지역']
target_col = '등록차량수'

In [ ]:
!pip uninstall pycaret -y

!pip install pycaret[full] ngboost shap

     |████████████████████████████████| 266kB 29.2MB/s 
     |████████████████████████████████| 358kB 39.4MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 112kB 50.1MB/s 
     |████████████████████████████████| 256kB 34.3MB/s 
     |████████████████████████████████| 1.7MB 33.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.0MB 34.8MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 6.8MB 31.9MB/s 
     |████████████████████████████████| 174kB 43.8MB/s 
     |████████████████████████████████| 276kB 41.4MB/s 
     |████████████████████████████████| 14.2MB 140kB/s 
     |████████████████████████████████| 1.4MB 28.0MB/s 
     |████████████████████████████████| 49.4MB 50kB/s 
     |████████████████████████████████| 3.6MB 33.1MB

In [ ]:
train_agg.columns

Index(['총세대수', '지역', '공가수', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수', '임대건물구분_상가', '임대건물구분_아파트',
       '공급유형_공공임대(50년)', '공급유형_공공임대(5년/10년/분납/분양)', '공급유형_국민임대/장기전세',
       '공급유형_영구임대', '공급유형_임대상가', '공급유형_행복주택', '자격유형_A', '자격유형_D',
       '자격유형_국민임대/장기전세_공급대상', '자격유형_영구임대_공급대상', '자격유형_행복주택_공급대상',
       '지역별차량등록대수(연령분포고려)', '단지코드내차량등록대수'],
      dtype='object')

In [ ]:
# Compare Models
from pycaret.classification import *
from pycaret.regression import *
from pycaret.utils import check_metric

FEATS = ['총세대수', '지역', '공가수', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수', '임대건물구분_상가', '임대건물구분_아파트',
       '공급유형_공공임대(50년)', '공급유형_공공임대(5년/10년/분납/분양)', '공급유형_국민임대/장기전세',
       '공급유형_영구임대', '공급유형_임대상가', '공급유형_행복주택', '자격유형_A', '자격유형_D',
       '자격유형_국민임대/장기전세_공급대상', '자격유형_영구임대_공급대상', '자격유형_행복주택_공급대상',
       '지역별차량등록대수(연령분포고려)', '단지코드내차량등록대수']
# categorical_features = ['지역']
# numeric_features = [c for c in FEATS if c not in [target_col]+categorical_features]
categorical_features = []
numeric_features = []

reg1 = setup(data = train_agg[FEATS], 
             train_size=0.9,  
             target = '등록차량수', 
             session_id=seed,
             categorical_features = categorical_features,
             numeric_features = numeric_features ,             
             )
top5 = compare_models(n_select = 5, sort = 'MAE')
print('\n'.join([str(t) for t in top5]))
df_top_by_comparing = pull()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,126.0136,3.176651e+04,176.2453,0.7695,0.4267,0.3765,0.018
ridge,Ridge Regression,127.3502,3.219914e+04,177.4537,0.7674,0.4749,0.3970,0.014
llar,Lasso Least Angle Regression,127.3556,3.325250e+04,180.3429,0.7584,0.4069,0.3863,0.018
lr,Linear Regression,128.4673,3.288001e+04,179.3238,0.7619,0.4853,0.4025,0.015
catboost,CatBoost Regressor,130.7232,3.736707e+04,189.9629,0.7393,0.4029,0.3593,2.294
gbr,Gradient Boosting Regressor,131.4920,3.588848e+04,186.4765,0.7421,0.4146,0.3639,0.085
omp,Orthogonal Matching Pursuit,132.4994,3.368691e+04,181.7300,0.7541,0.4153,0.3912,0.016
rf,Random Forest Regressor,132.6098,3.743328e+04,189.8867,0.7357,0.3845,0.3492,0.527
lightgbm,Light Gradient Boosting Machine,133.3367,3.818191e+04,191.5537,0.7278,0.4009,0.3523,0.036
en,Elastic Net,133.4587,3.610493e+04,187.2142,0.7415,0.4727,0.4081,0.016


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=47,
      selection='cyclic', tol=0.0001, warm_start=False)
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=47, solver='auto', tol=0.001)
LassoLars(alpha=1.0, copy_X=True, eps=2.220446049250313e-16, fit_intercept=True,
          fit_path=True, jitter=None, max_iter=500, normalize=True,
          positive=False, precompute='auto', random_state=47, verbose=False)
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)


In [ ]:
# Compare Models
from pycaret.classification import *
from pycaret.regression import *
from pycaret.utils import check_metric

FEATS = ['총세대수', '지역', '공가수', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수', '임대건물구분_상가', '임대건물구분_아파트',
       '공급유형_공공임대(50년)', '공급유형_공공임대(5년/10년/분납/분양)', '공급유형_국민임대/장기전세',
       '공급유형_영구임대', '공급유형_임대상가', '공급유형_행복주택', '자격유형_A', '자격유형_D',
       '자격유형_국민임대/장기전세_공급대상', '자격유형_영구임대_공급대상', '자격유형_행복주택_공급대상']

categorical_features = []
numeric_features = []

reg1 = setup(data = train_agg[FEATS], 
             train_size=0.9,  
             target = '등록차량수', 
             session_id=seed,
             categorical_features = categorical_features,
             numeric_features = numeric_features ,             
             )
top5 = compare_models(n_select = 5, sort = 'MAE')
print('\n'.join([str(t) for t in top5]))
df_top_by_comparing = pull()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,125.8689,3.145419e+04,175.2943,0.7731,0.4452,0.3831,0.016
lasso,Lasso Regression,126.4382,3.174636e+04,176.1957,0.7695,0.4218,0.3778,0.015
ridge,Ridge Regression,127.3426,3.202328e+04,176.9684,0.7679,0.4765,0.4024,0.014
llar,Lasso Least Angle Regression,127.3522,3.325125e+04,180.3386,0.7584,0.4069,0.3862,0.016
huber,Huber Regressor,128.4279,3.433354e+04,182.9095,0.7548,0.4224,0.3590,0.044
lr,Linear Regression,128.9851,3.290567e+04,179.2391,0.7606,0.4671,0.4104,0.292
en,Elastic Net,132.3368,3.561246e+04,185.9203,0.7447,0.4712,0.4071,0.016
omp,Orthogonal Matching Pursuit,132.4994,3.368691e+04,181.7300,0.7541,0.4153,0.3912,0.015
catboost,CatBoost Regressor,132.7945,3.694632e+04,190.0083,0.7392,0.4040,0.3664,2.007
lightgbm,Light Gradient Boosting Machine,133.8985,3.831522e+04,192.8646,0.7243,0.4088,0.3632,0.081


BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False)
Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=47,
      selection='cyclic', tol=0.0001, warm_start=False)
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=47, solver='auto', tol=0.001)
LassoLars(alpha=1.0, copy_X=True, eps=2.220446049250313e-16, fit_intercept=True,
          fit_path=True, jitter=None, max_iter=500, normalize=True,
          positive=False, precompute='auto', random_state=47, verbose=False)
HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, max_iter=100,
               tol=1e-05, warm_start=False)


In [ ]:
# df_top_by_comparing.to_csv('./output_pycaret/00_compare_models.csv',index=True)
df_top_by_comparing.index

Index(['lasso', 'ridge', 'llar', 'lr', 'catboost', 'gbr', 'omp', 'rf',
       'lightgbm', 'en', 'et', 'br', 'huber', 'xgboost', 'ada', 'knn', 'dt',
       'par', 'lar'],
      dtype='object')

In [ ]:
# 모델들, grid score df, plot 등이 모두 저장되는 상위경로
now_time = datetime.now(timezone(timedelta(hours=9))).strftime('%m%d_%H%M')
experiment_name = '_' + 'LB111수정_lasso_외부데이터활용_단지코드별&지역별차량등록대수_변수추가'

now_time += experiment_name

upper_dir_of_pycaret = f'/content/drive/MyDrive/dacon_parking_data/output_pycaret/{now_time}'
print(f"upper_dir_of_pycaret : {upper_dir_of_pycaret}")

fold=10
# FEATS = [c for c in final_df.columns if c not in ['단지코드']]
target_feature = '등록차량수'

print(f'FEATS : {len(FEATS)}')
print(f"target : {target_feature}")
print(f'cat_features : {len(categorical_features)}')
print(f'numeric_features : {len(numeric_features)}')

upper_dir_of_pycaret : /content/drive/MyDrive/dacon_parking_data/output_pycaret/0630_0059_LB111수정_lasso_외부데이터활용_단지코드별&지역별차량등록대수_변수추가
FEATS : 22
target : 등록차량수
cat_features : 0
numeric_features : 0


In [ ]:
copy_train = train[['단지코드','등록차량수']].drop_duplicates().reset_index(drop=True)
no_nan_danjicode_idxes = copy_train.loc[copy_train['단지코드'].isin(no_nan_danjicode),:].index

In [ ]:
holdout = train_agg[FEATS].iloc[no_nan_danjicode_idxes]
holdout.shape

(378, 22)

# Training

In [ ]:
n_iter = 20

In [ ]:
settings = setup(
                data=train_agg[FEATS], 
                target=target_feature, 
                train_size=0.95, 
                categorical_features = categorical_features,
                numeric_features = numeric_features ,                        
                data_split_shuffle = True, # True가 default
                session_id = seed,
                # fold_strategy = last_fold,
                # experiment_name=experiment_name,
                # log_experiment=False,   # compare_model 할 때에는 하이퍼파라미터가 어떻게 튜닝되었는지 볼 수 있어서 유용하다고 한다.
                )

# from ngboost import NGBClassifier
# ngc = NGBClassifier()
# ngboost = create_model(ngc)

model_name_list=['lasso']   # 'et', 'catboost', 'rf', 'gbr', 'llar', 'omp', 'en', 'ada', 'br', 'lr', 'lasso', 'dt', 'ridge',
ensemble_method = 'blend_models'  # 'blend_models', 'stack_models' ->  (Regression : 'create_stacknet'), (Classifier : 'stack_models'),
meta_model_for_stack = 'xgboost' # If using stack_models

,Description,Value
0,session_id,47
1,Target,등록차량수
2,Original Data,"(423, 22)"
3,Missing Values,False
4,Numeric Features,8
5,Categorical Features,13
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(401, 33)"


In [ ]:
## Classification Models :  https://github.com/pycaret/pycaret/blob/master/pycaret/containers/models/classification.py
## Regression Modelss : https://github.com/pycaret/pycaret/blob/master/pycaret/containers/models/regression.py

## 각 단계마다 (create - tune - ensemble - finalize) 모델, 정보, plot 저장해주는 코드

def plot_save_pycaret_model(name, model,mode='tune'):
    # Feature Importance Plot 저장하기
    plot_dir = f'{upper_dir_of_pycaret}/plot_result'
    os.makedirs(plot_dir, exist_ok=True)
    os.chdir(plot_dir)
    # top10 feature
    feature_importance_plot = plot_model(model, plot = 'feature', save=True)
    os.rename(f'{plot_dir}/Feature Importance.png',f'{plot_dir}/{mode}_{name}_10_feature_importance.png')
    # all feature
    feature_importance_plot = plot_model(model, plot = 'feature_all', save=True)
    os.rename(f'{plot_dir}/Feature Importance (All).png',f'{plot_dir}/{mode}_{name}_all_feature_importance.png')

# def create_and_save_pycaret_model(name):
#     created_model = create_model(name, cross_validation = True)
#     # Display되는 score grid dataframe도 가져올 수가 있다. 
#     df_result_model = pull()
#     df_result_model.to_csv(f'{upper_dir_of_pycaret}/{name}_create_grid_df.csv')
#     # 실험결과를 저장하는 것도 가능하다.
#     save_experiment(f'{upper_dir_of_pycaret}/{name}_create_exp_result')
#     # 모델을 저장하는 것도 가능하다.
#     save_model(created_model, f'{upper_dir_of_pycaret}/{name}_model_saved')
#     return created_model

def tune_and_save_pycaret_model(name, model, optimize = 'MAE', fold = fold, n_iter = n_iter):
    print('Now Tuning the models....')
    tuned_model = tune_model(model, optimize = optimize, fold = fold, n_iter = n_iter)   # 
    # Display되는 score grid dataframe도 가져올 수가 있다. 
    df_result_model = pull()
    os.makedirs(f"{upper_dir_of_pycaret}/grid_df",exist_ok=True)
    df_result_model.to_csv(f'{upper_dir_of_pycaret}/grid_df/{name}_tuned_grid_df.csv')
    # 모델의 각종 plot을 저장하는 것도 가능하다.
    plot_save_pycaret_model(name, tuned_model, mode='tune')
    # 실험결과를 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/expr",exist_ok=True)
    try:
        save_experiment(f'{upper_dir_of_pycaret}/expr/{name}_tuned_exp_result')
    except:
        print('tuned model의 실험결과는 저장할 수 없습니다.')
    # 모델을 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/models",exist_ok=True)
    save_model(tuned_model, f'{upper_dir_of_pycaret}/models/{name}_tuned_model_saved')
    return tuned_model

def ensemble_and_save_pycaret_model(
                                    model_name_list,
                                    estimator_list,
                                    fold = fold,
                                    optimize = 'MAE',
                                    meta_model = None,
                                    mode = None,
                                    ):
    print('Now Ensemble the models....')
    if mode=='blend_models':
        ensembled_model = blend_models(estimator_list = estimator_list, fold = fold, optimize = optimize)
    elif mode=='stack_models':
        ensembled_model = stack_models(estimator_list = estimator_list, fold = fold, optimize = optimize, meta_model = meta_model)
    # Display되는 score grid dataframe도 가져올 수가 있다. 
    df_result_model = pull()
    name = '_'.join(model_name_list)
    os.makedirs(f"{upper_dir_of_pycaret}/grid_df",exist_ok=True)
    df_result_model.to_csv(f'{upper_dir_of_pycaret}/grid_df/{name}_ensembled_grid_df.csv')
    # 모델의 각종 plot을 저장하는 것도 가능하다.
    try:
        plot_save_pycaret_model(name, ensembled_model, mode='ensemble')
    except:
        print(f'ensemble model은 변수중요도 plot을 저장할 수 없습니다')
    # 실험결과를 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/expr",exist_ok=True)
    try:
        save_experiment(f'{upper_dir_of_pycaret}/expr/{name}_ensembled_exp_result')
    except:
        print(f'ensemble model은 실험결과를 저장할 수 없습니다')
    # 모델을 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/models",exist_ok=True)
    save_model(ensembled_model, f'{upper_dir_of_pycaret}/models/{name}_ensembled_model_saved')
    return ensembled_model

def finalize_and_save_pycaret_model(model_name_list,model,):
    print('Now Finalizing the model....')
    finalized_model = finalize_model(model)
    # Display되는 score grid dataframe도 가져올 수가 있다. 
    df_result_model = pull()
    name = '_'.join(model_name_list)
    os.makedirs(f"{upper_dir_of_pycaret}/grid_df",exist_ok=True)
    df_result_model.to_csv(f'{upper_dir_of_pycaret}/grid_df/{name}_finalized_grid_df.csv')
    # 모델의 각종 plot을 저장하는 것도 가능하다.
    try:
        plot_save_pycaret_model(name, finalized_model, mode='finalize')
    except:
        print(f'final model은 변수중요도 plot을 저장할 수 없습니다')
    # 실험결과를 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/expr",exist_ok=True)
    try:
        save_experiment(f'{upper_dir_of_pycaret}/expr/{name}_finalized_exp_result')
    except:
        print(f'final model은 실험결과를 저장할 수 없습니다')
    # 모델을 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/models",exist_ok=True)
    save_model(finalized_model, f'{upper_dir_of_pycaret}/models/{name}_finalized_model_saved')
    return finalized_model


In [ ]:
def ensemble_automl(
                    final_df,
                    holdout,
                    FEATS,
                    cat_features=[],
                    continuous_features=[],
                    seed=seed,
                    model_name_list = model_name_list,
                    ensemble_method = ensemble_method,
                    meta_model_for_stack = meta_model_for_stack,
                    settings = settings,
                    ):

    # final_df, holdout = datasets
    random.seed(seed)
    # fold_strategy = KFold(n_splits=10, shuffle=True, random_state=42)
    settings = settings

    # train_size만큼을 가지고 선언된 model_name_list 모델들을 학습을 함
    # create_model(name, sort='MAE', cross_validation = True)
    models_before_tune = [
                            create_model(
                                name, 
                                fold=fold,
                                cross_validation = True,
                                # cross_validation = False,
                                ) 
                            for name in model_name_list]
    # 앞서만든 모델들을 train_size만큼 가지고 튜닝함 (n_iter만큼 AutoML)
    # models_after_tune = [tune_model(model, optimize = 'MAE', fold = 10, n_iter = 10) for model in models_before_tune]
    models_after_tune = [tune_and_save_pycaret_model(name, model, optimize = 'MAE', fold = fold, n_iter = 10) for name, model in zip(model_name_list, models_before_tune)]


    # 튜닝된 모델들을 train_size만큼 가지고 앙상블
    if len(models_after_tune)<=1:
        ensembled = models_after_tune[0]
    else:
        ensembled = ensemble_and_save_pycaret_model(model_name_list,estimator_list=models_after_tune,fold = fold,optimize = 'MAE',method = method,meta_model = meta_model_for_stack,mode = ensemble_method)
    # elif ensemble_method=='blend_models':
    #     ensembled = blend_models(estimator_list = models_after_tune, fold = 10, method = 'soft', optimize = 'MAE')
    # elif ensemble_method=='stack_models':
    #     ensembled = stack_models(estimator_list = models_after_tune, meta_model = meta_model_for_stack, fold = 10, optimize = 'MAE')

    # 마지막 학습(Finalize)
    # 앞서 앙상블된 모델을 => setup으로 나눠져 쓰지않았던 valid까지 포함된 100퍼센트를 사용하여 fitting함
    final_model = finalize_and_save_pycaret_model(model_name_list,ensembled)

    metric_result = []
    prediction = predict_model(final_model, data=holdout[FEATS], )  # raw_score = True
    # df_holdout_score = pull()

    # os.makedirs(f"{upper_dir_of_pycaret}/holdout_score",exist_ok=True)
    # df_holdout_score.to_csv(f'{upper_dir_of_pycaret}/holdout_score/finalize_holdout_score.csv')

    metric_result.append(f"HoldOut 데이터 MAE = {check_metric(prediction['등록차량수'], prediction['Label'], metric = 'MAE')}")
    return final_model, metric_result


In [ ]:
final_model, metric_result = ensemble_automl(
                                            train_agg,
                                            holdout,
                                            FEATS,
                                            categorical_features,
                                            numeric_features,
                                            seed=seed,
                                            model_name_list=model_name_list,
                                            ensemble_method = ensemble_method,
                                            meta_model_for_stack = meta_model_for_stack,
                                            )

print(final_model)

print(f"Error : ",'\n'.join(metric_result))

# BayesianRidge(alpha_1=1e-06, alpha_2=0.15, alpha_init=None, compute_score=False,
#               copy_X=True, fit_intercept=False, lambda_1=0.1, lambda_2=0.15,
#               lambda_init=None, n_iter=300, normalize=False, tol=0.001,
#               verbose=False)
# ACC & AUC :  HoldOut 데이터 MAE: 120.8852

# BayesianRidge(alpha_1=1e-06, alpha_2=0.15, alpha_init=None, compute_score=False,
#               copy_X=True, fit_intercept=False, lambda_1=0.1, lambda_2=0.15,
#               lambda_init=None, n_iter=300, normalize=False, tol=0.001,
#               verbose=False)
# ACC & AUC :  HoldOut 데이터 MAE: 120.7506

final model은 실험결과를 저장할 수 없습니다
Transformation Pipeline and Model Succesfully Saved
Lasso(alpha=1.73, copy_X=True, fit_intercept=False, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=47,
      selection='cyclic', tol=0.0001, warm_start=False)
Error :  HoldOut 데이터 MAE = 121.9396


In [ ]:
# 이번 Experiment Arg 관리
config_key = [
            'now_time',
            'seed','fold','model_name_list','ensemble_method','meta_model_for_stack',
            'categocategorical_features','numeric_features','FEATS','LENGTH_FEATS','n_iter','metric_result'
            ]
config_value = [
                now_time,
                seed, fold, model_name_list, ensemble_method, meta_model_for_stack, 
             categorical_features, numeric_features, FEATS,len(FEATS),n_iter,metric_result,]


In [ ]:
cd /content/drive/MyDrive/dacon_parking_data

/content/drive/MyDrive/dacon_parking_data


## Submission

In [ ]:
print(train_agg.shape)
print(test_agg.shape)

(423, 22)
(150, 21)


In [ ]:
print(train_agg['등록차량수'].max())
print(train_agg['등록차량수'].min())

2550.0
13.0


In [ ]:
str_model = str(final_model)
str_model

"Lasso(alpha=1.73, copy_X=True, fit_intercept=False, max_iter=1000,\n      normalize=False, positive=False, precompute=False, random_state=47,\n      selection='cyclic', tol=0.0001, warm_start=False)"

In [ ]:
sample_submission = pd.read_csv('./data/sample_submission.csv')
# MAKE PREDICTION
prediction = predict_model(final_model, data=test_agg[[f for f in FEATS if f not in ['등록차량수']]])
total_preds = prediction["Label"]
total_preds_dict = total_preds.to_dict()
sample_submission['num'] = total_preds.values

# SAVE OUTPUT
prediction_name = f"{now_time}_NFeatures-{len(FEATS)}" # 

output_dir = f'{upper_dir_of_pycaret}/코드공유_submission'
os.makedirs(output_dir, exist_ok=True)    
write_path = os.path.join(output_dir, f"{prediction_name}.csv")
print("Saving Final Output CSVs...")
sample_submission.to_csv(f'{write_path}',index=False)
print(f"writing csv : {write_path}")

import json
# Save Config
write_path = os.path.join(upper_dir_of_pycaret, f"{prediction_name}_config.json")
config_dict = {k:v for k,v in zip(config_key, config_value)}
print("Saving Final Config Dict...")
## json파일 저장 ##
with open(write_path, "w") as fp:
    print(f"writing config : {write_path}")
    json.dump(config_dict, fp, indent=4)


Saving Final Output CSVs...
writing csv : /content/drive/MyDrive/dacon_parking_data/output_pycaret/0630_0059_LB111수정_lasso_외부데이터활용_단지코드별&지역별차량등록대수_변수추가/코드공유_submission/0630_0059_LB111수정_lasso_외부데이터활용_단지코드별&지역별차량등록대수_변수추가_NFeatures-22.csv
Saving Final Config Dict...
writing config : /content/drive/MyDrive/dacon_parking_data/output_pycaret/0630_0059_LB111수정_lasso_외부데이터활용_단지코드별&지역별차량등록대수_변수추가/0630_0059_LB111수정_lasso_외부데이터활용_단지코드별&지역별차량등록대수_변수추가_NFeatures-22_config.json


In [ ]:
print(total_preds.max())
print(total_preds.min())

1601.8927001953125
7.622137069702148


In [ ]:
sample_submission

,code,num
0,C1072,660.792542
1,C1128,1179.293823
2,C1456,511.687286
3,C1840,538.658325
4,C1332,1031.107422
...,...,...
145,C2456,205.214050
146,C1266,377.961090
147,C2152,7.622137
148,C1267,205.927429
